# Playing around with the Spotify API

In [46]:
import pandas as pd
import numpy as np
import os
import sys
import spotipy
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from scipy import spatial

In [5]:
'''
define the API connection - usually log in from here as well...
'''
token = util.prompt_for_user_token('', '')

client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


#spotify=spotipy.Spotify()




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        


In [7]:
def get_uri(artist):
    results = sp.search(q = "artist:" + artist, type = "artist")
    artist_id = results['artists']['items'][0]['uri'] # changed from '1' to '0' ??
   
    return(artist_id)

get_uri("Beyonce")

'spotify:artist:6vWDO969PvNqNYHIOW5v0m'

In [9]:
def related_artists(name):
    related = sp.artist_related_artists(get_uri(name))
    rels = []
    for artist in related['artists']:
        rels.append(artist['name'])
    return(rels)


def next_artist(artist, artists = []):
    ''' artists is the storage of past people that have been in there'''
    RelatedArtists = related_artists(artist)
    relates = [] # empty list to append with future 
    
    for i in RelatedArtists:
        relates.append(related_artists(i) )
    
    # create a bag of words - to index each artist in:
    
    BOW = np.unique(relates).flatten() 
    vecs = np.zeros((len(relates), len(BOW)))  # unique artists as columns...

    for i in range(len(relates)):
        # turn each into a vector of 0 and 1
        for art in relates[i]:
            # this loops over each artist:
            idx = np.where(BOW == art)[0][0]
            # set that idx number = to 1
            vecs[i,idx] = 1
    
    
    # then calculate cosine similarity and select the lowest similarity artist and print them
    sims = np.zeros((len(relates),len(relates)))
    sim_means = []
    for i in range(len(vecs)):
        # loop over each vector
        for j in range(len(vecs)):
            # do this again because you want to get similarity between each matrix...
            sims[i,j] = 1 - spatial.distance.cosine(vecs[i],vecs[j])
        # calculate row wise means:
        sim_means.append(np.var(sims[i] ))
    
    # find the lowest scoring similarity artist
    lowest_idx = np.where(sim_means == max(sim_means) )[0][0]
    #print(lowest_idx)
    # return artists name:
    

    #if (RelatedArtists[lowest_idx] in artists) == True :
    # do the process again...
    
    
    print(RelatedArtists[lowest_idx])
    
    
    return(RelatedArtists[lowest_idx])

In [12]:
# import some data to autoencode -- top artists
os.chdir('C:\\Users\\Max.Polaczuk\\Google Drive\\music_merge\\artists')
pop50 = pd.read_csv('pop50.csv')

In [16]:
print(pop50.ix[:,0].tolist())
pop = (pop50.ix[:,0].tolist() )

['THE BEATLES', 'MADONNA', 'ELTON JOHN', 'ELVIS PRESLEY', 'MARIAH CAREY', 'STEVIE WONDER', 'JANET JACKSON', 'MICHAEL JACKSON', 'WHITNEY HOUSTON', 'THE ROLLING STONES', 'PAUL MCCARTNEY', 'BEE GEES', 'RIHANNA', 'USHER', 'CHICAGO', 'THE SUPREMES', 'DARYL HALL JOHN OATES', 'PRINCE', 'ROD STEWART', 'OLIVIA NEWTON-JOHN', 'ARETHA FRANKLIN', 'MARVIN GAYE', 'PHIL COLLINS', 'KATY PERRY', 'BILLY JOEL', 'DIANA ROSS', 'THE 4 SEASONS', 'THE TEMPTATIONS', 'DONNA SUMMER', 'THE BEACH BOYS', 'LIONEL RICHIE', 'NEIL DIAMOND', 'CARPENTERS', 'TAYLOR SWIFT', 'BOYZ II MEN', 'THE JACKSONS', 'CONNIE FRANCIS', 'KENNY ROGERS', 'BEYONCE', 'BRENDA LEE', 'BARBRA STREISAND', 'BRYAN ADAMS', 'CHER', 'MAROON 5', 'GEORGE MICHAEL', 'THE BLACK EYED PEAS', 'BOBBY VINTON', 'JOHN MELLENCAMP', 'THREE DOG NIGHT', 'HUEY LEWIS & THE NEWS']


In [ ]:
## for each artist get their uid and save

uris = []

for p in pop:
    uris.append(get_uri(p))
    

In [ ]:
## now we can use these fellows to return their artist information:

# save each genre in a list 
genres = []
for i in range(len(uris)):
    K = sp.artist(uris[i])['genres']
    for k in range(len(K)):
        genres.append(K[k])



In [25]:
# get the unique list of genres and index them..
genres = np.unique(genres)
print(genres)

['adult standards' 'album rock' 'australian dance' 'bachata' 'blues-rock'
 'boy band' 'brill building pop' 'british blues' 'british invasion'
 'bubblegum pop' 'cabaret' 'canadian pop' 'chicago soul' 'christmas'
 'classic funk rock' 'classic rock' 'country' 'country christmas'
 'country rock' 'dance pop' 'dance rock' 'deep indie r&b' 'deep pop r&b'
 'disco' 'doo-wop' 'easy listening' 'electric blues' 'escape room'
 'europop' 'folk' 'folk christmas' 'folk rock' 'funk' 'funk rock'
 'glam rock' 'hard rock' 'hip hop' 'hip pop' 'hollywood' 'indie r&b'
 'jazz blues' 'latin' 'latin hip hop' 'latin pop' 'lounge' 'mellow gold'
 'memphis soul' 'merengue urbano' 'merseybeat' 'motown' 'nashville sound'
 'neo mellow' 'neo soul' 'new jack swing' 'new romantic' 'new wave'
 'new wave pop' 'opera' 'operatic pop' 'outlaw country' 'piano rock' 'pop'
 'pop christmas' 'pop rap' 'pop rock' 'post-teen pop' 'protopunk'
 'psychedelic rock' 'quiet storm' 'r&b' 'rap' 'reggaeton' 'rock'
 'rock-and-roll' 'rockabill

In [32]:
# setup null matrix length of artists by length of genres...
GEN = np.zeros((len(pop),len(genres)))

# run a loop over each artist to get their vector of 

for i in range(len(uris)):
    K = sp.artist(uris[i])['genres']
    for k in range(len(K)):
        idx = np.where(genres == K[k])[0][0]
        GEN[i,idx] = 1

In [33]:
print(GEN)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [38]:
# now make dataframe of artists and their genres:

frame = pd.DataFrame(GEN)
frame['Artist'] = pop
frame['URI'] = uris

cols = frame.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]

In [44]:
frame = frame[cols]
print(frame)
frame.to_csv('Artist_genre.csv')

inps = np.array(GEN)

                   Artist                                    URI    0    1  \
0             THE BEATLES  spotify:artist:3WrFJ7ztbogyGnTHbHJFl2  0.0  0.0   
1                 MADONNA  spotify:artist:6tbjWDEIzxoDsBA1FuhfPW  0.0  0.0   
2              ELTON JOHN  spotify:artist:3PhoLpVuITZKcymswpck5b  0.0  1.0   
3           ELVIS PRESLEY  spotify:artist:43ZHCT0cAZBISjO8DG9PnE  0.0  0.0   
4            MARIAH CAREY  spotify:artist:4iHNK0tOyZPYnBU7nGAgpQ  0.0  0.0   
5           STEVIE WONDER  spotify:artist:7guDJrEfX3qb6FEbdPA5qi  1.0  0.0   
6           JANET JACKSON  spotify:artist:4qwGe91Bz9K2T8jXTZ815W  0.0  0.0   
7         MICHAEL JACKSON  spotify:artist:3fMbdgg4jU18AjLCKBhRSm  0.0  0.0   
8         WHITNEY HOUSTON  spotify:artist:6XpaIBNiVzIetEPCWDvAFP  0.0  0.0   
9      THE ROLLING STONES  spotify:artist:22bE4uQ6baNwSHPVcDxLCe  0.0  1.0   
10         PAUL MCCARTNEY  spotify:artist:4STHEaNw4mPZ2tzheohgXB  0.0  1.0   
11               BEE GEES  spotify:artist:1LZEQNv7sE11VDY3SdxQeN

In [47]:
# RUN an autoencoder in 3D over this...

''' define the model architecture '''

inputs = Input(shape=(np.shape(inps)[1],))

encoding_dim = 3
# a layer instance is callable on a tensor, and returns a tensor
ENCOD = Dense(encoding_dim, activation='relu')(inputs)
DECOD = Dense(np.shape(inps)[1], activation='relu')(ENCOD)
#predictions = Dense(10, activation='softmax')(x)

# this creates a model that includes
# the Input layer and three Dense layers
autoencoder = Model(input=inputs, output=DECOD)

# this model maps an input to its encoded representation
encoder = Model(inputs, output = ENCOD)

# create a placeholder for an encoded (N-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
#decoder_layer = autoencoder.layers[-1]
# create the decoder model
#decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

### compile the model...
autoencoder.compile(optimizer='rmsprop', loss='mse')

In [89]:
autoencoder.fit(inps, inps,
                nb_epoch=10,batch_size=100,shuffle=True )

Epoch 1/10
50/50 [==============================] - 0s - loss: 0.0443
Epoch 2/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 3/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 4/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 5/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 6/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 7/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 8/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 9/10
50/50 [==============================] - 0s - loss: 0.0442
Epoch 10/10
50/50 [==============================] - 0s - loss: 0.0442


In [90]:
''' save the predictions... '''
encoded_inputs = pd.DataFrame(encoder.predict(inps))

In [91]:
''' add in names and spotify URIs to the df above... '''

encoded_inputs['Artist'] = pop
encoded_inputs['URI'] = uris

cols = encoded_inputs.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]

encoded_inputs.to_csv('encoded_artists.csv')